In [216]:
import sys
import os

# os.getcwd() => '/home/genie/Documents/Projekti/cash-flow/jupyter'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Moves up one level to 'cash-flow'
# project_root = '/home/genie/Documents/Projekti/cash-flow'

# Add the module Converters directory to sys.path
sys.path.append(os.path.join(project_root, 'cash_flow', 'util'))



In [217]:
# Imports and engine

import pandas as pd
import numpy as np
import datetime
from sqlalchemy import create_engine

from Converters import date_format

pd.options.mode.copy_on_write = True
engine = create_engine("sqlite:///../data/database.db", echo=False)

In [218]:
# -------------------
# CONFIGURABLE INPUTS
# -------------------

date_from = "2025-06-01"
date_through = "2025-12-31"

# Choose period granularity: "day", "week", "month", "quarter", "year"
period = "month"


In [219]:
# Mapping period to offset (end of period)
period_offsets = {
    "day": pd.offsets.Day(0),
    "week": pd.offsets.Week(weekday=6),         # Sunday
    "month": pd.offsets.MonthEnd(0),
    "quarter": pd.offsets.QuarterEnd(startingMonth=12),
    "year": pd.offsets.YearEnd(0)
}
date_offset = period_offsets.get(period, pd.offsets.MonthEnd(0))
date_freq = {
    "day": "D",
    "week": "W-SUN",         # Sunday
    "month": "ME",
    "quarter": "QE",
    "year": "YE"
}
date_frequency = date_freq.get(period, "ME")

In [220]:
# Load all data from database

actual = pd.read_sql_query('SELECT * FROM G10_CashFlow_Actual_Corresponding WHERE d_date <= "' + date_through + '" ', engine)
pending = pd.read_sql_query('SELECT * FROM G12_CashFlow_Pending_Corresponding WHERE p_date <= "' + date_through + '" ', engine)
budgeted = pd.read_sql_query('SELECT * FROM F01_BudgetEntries WHERE date >= "' + date_from + '" AND date <= "' + date_through + '" ', engine)
cash = pd.read_sql_query('SELECT * FROM G02_CashTransactions WHERE d_date <= "' + date_through + '" ', engine)
definition_df = pd.read_sql_table("E01_CashFlowDefinition", engine)

definition_accounts_df = pd.read_sql_table("E01_CashFlowDefinitionAccounts", engine)
definition_totals_df = pd.read_sql_table("E01_CashFlowDefinitionTotals", engine)



In [221]:
# Ensure required columns and formats

actual['period_end'] = pd.to_datetime(actual['d_date'], format='mixed', errors='coerce').apply(lambda d: date_offset.rollforward(d) if pd.notnull(d) else pd.NaT)
actual['cf_amount'] = np.where(actual['gl_entry_type'] == 'CR', actual['gl_amount_LC'], -actual['gl_amount_LC'])
pending['period_end'] = pd.to_datetime(pending['p_date'], format='mixed', errors='coerce').apply(lambda d: date_offset.rollforward(d) if pd.notnull(d) else pd.NaT)
pending['cf_amount'] = np.where(pending['gl_entry_type'] == 'CR', pending['gl_amount_LC'], -pending['gl_amount_LC'])
budgeted['period_end'] = pd.to_datetime(budgeted['date'], format='mixed', errors='coerce').apply(lambda d: date_offset.rollforward(d) if pd.notnull(d) else pd.NaT)
budgeted['cf_amount'] = np.where(budgeted['cash_type'] == 'Receipt', budgeted['amount_LC'], -budgeted['amount_LC'])
cash['period_end'] = pd.to_datetime(cash['d_date'], format='mixed', errors='coerce').apply(lambda d: date_offset.rollforward(d) if pd.notnull(d) else pd.NaT)
cash['cf_amount'] = np.where(cash['gl_entry_type'] == 'DR', cash['gl_amount_LC'], -cash['gl_amount_LC'])

definition_df.rename(columns={"id":"definition_id"}, inplace=True)
definition_acc_df = definition_df[definition_df["definition_type"] == 1]
definition_tot_df = definition_df[definition_df["definition_type"] == 2]
definition_bal_df = definition_df[definition_df["definition_type"] == 3]

In [222]:
actual.head()

,d_id,cash_status,cash_type,d_type,d_date,d_number,d_customer_id,d_vendor_ir,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC,period_end,cf_amount
0,2390,Actual,Receipt,1,2020-01-01 00:00:00.000000,1890,78.0,NaN,Payment 1890,EUR,CR,5721,202.78,202.78,2020-01-31,202.78
1,2390,Actual,Receipt,1,2020-01-01 00:00:00.000000,1890,78.0,NaN,Payment 1890,EUR,CR,6550,965.64,965.64,2020-01-31,965.64
2,2340,Actual,Receipt,1,2020-01-03 00:00:00.000000,1840,24.0,NaN,Payment 1840,EUR,CR,5721,1346.57,1346.57,2020-01-31,1346.57
3,2340,Actual,Receipt,1,2020-01-03 00:00:00.000000,1840,24.0,NaN,Payment 1840,EUR,CR,6550,6412.27,6412.27,2020-01-31,6412.27
4,4808,Actual,Payment,2,2020-01-04 00:00:00.000000,3508,NaN,9.0,Payment 3508,EUR,DR,5721,505.54,505.54,2020-01-31,-505.54


In [223]:
pending.head()

,d_id,cash_status,cash_type,d_type,p_date,d_number,d_customer_id,d_vendor_id,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC,period_end,cf_amount
0,8,Pending,Receipt,3,2025-05-24,8,39.0,NaN,Invoice 8,EUR,CR,6110,601.70,601.70,2025-05-31,601.70
1,8,Pending,Receipt,3,2025-05-24,8,39.0,NaN,Invoice 8,EUR,CR,5721,126.36,126.36,2025-05-31,126.36
2,11,Pending,Receipt,3,2025-05-24,11,49.0,NaN,Invoice 11,EUR,CR,6110,2833.28,2833.28,2025-05-31,2833.28
3,11,Pending,Receipt,3,2025-05-24,11,49.0,NaN,Invoice 11,EUR,CR,5721,594.99,594.99,2025-05-31,594.99
4,15,Pending,Receipt,3,2025-05-24,15,29.0,NaN,Invoice 15,EUR,CR,6550,8966.10,8966.10,2025-05-31,8966.10


In [224]:
budgeted.head()

,id,definition_id,cash_type,date,amount_LC,memo,period_end,cf_amount
0,5,2,Receipt,2025-06-01 00:00:00.000000,1000,Realizācija,2025-06-30,1000
1,6,2,Receipt,2025-06-08 00:00:00.000000,1000,Realizācija,2025-06-30,1000
2,7,2,Receipt,2025-06-15 00:00:00.000000,1000,Realizācija,2025-06-30,1000
3,8,2,Receipt,2025-06-22 00:00:00.000000,1000,Realizācija,2025-06-30,1000
4,9,2,Receipt,2025-06-29 00:00:00.000000,1000,Realizācija,2025-06-30,1000


In [225]:
# -------------------------------------------------------------------
# ************** (1) CashFlow based on accounts definitions *********
# -------------------------------------------------------------------

#  Prepare chart of CF definitions 

# Merge accounts into accounts definition
definition_accounts_df = pd.merge(
    definition_acc_df,
    definition_accounts_df,
    on="definition_id", how="left")
definition_accounts_df.drop(columns=["id"], inplace=True)

In [226]:
definition_accounts_df

,definition_id,key,definition_type,name,cash_type,account
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110
1,3,102,1.0,Maksājumi piegādātājiem,Payment,7110
2,3,102,1.0,Maksājumi piegādātājiem,Payment,7310
3,4,103,1.0,Maksājumi darbiniekiem,Payment,7210
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,6550
5,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,7510
6,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,1210
7,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,2310
8,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,5310
9,7,201,1.0,Izdevumi procentu maksājumiem,NaN,NaN


In [227]:
actual = pd.merge(definition_accounts_df,
    actual,
    left_on=['cash_type', 'account'],        # Columns in definition_df
    right_on=['cash_type', 'gl_account'], # Corresponding columns in transactions_df
    how='left'
)


In [228]:
actual.head()

,definition_id,key,definition_type,name,cash_type,account,d_id,cash_status,d_type,d_date,...,d_customer_id,d_vendor_ir,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC,period_end,cf_amount
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,1564.0,Actual,1.0,2020-01-09 00:00:00.000000,...,64.0,NaN,Payment 1064,EUR,CR,6110,4869.02,4869.02,2020-01-31,4869.02
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,2252.0,Actual,1.0,2020-01-10 00:00:00.000000,...,85.0,NaN,Payment 1752,EUR,CR,6110,2864.15,2864.15,2020-01-31,2864.15
2,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,1944.0,Actual,1.0,2020-01-12 00:00:00.000000,...,82.0,NaN,Payment 1444,EUR,CR,6110,2752.01,2752.01,2020-01-31,2752.01
3,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,2410.0,Actual,1.0,2020-01-16 00:00:00.000000,...,61.0,NaN,Payment 1910,EUR,CR,6110,4011.51,4011.51,2020-01-31,4011.51
4,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,1549.0,Actual,1.0,2020-01-17 00:00:00.000000,...,61.0,NaN,Payment 1049,EUR,CR,6110,3624.35,3624.35,2020-01-31,3624.35


In [229]:
pending = pd.merge(definition_accounts_df,
    pending,
    left_on=['cash_type', 'account'],        # Columns in definition_df
    right_on=['cash_type', 'gl_account'], # Corresponding columns in transactions_df
    how='left'
)


In [230]:
pending.head()

,definition_id,key,definition_type,name,cash_type,account,d_id,cash_status,d_type,p_date,...,d_customer_id,d_vendor_id,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC,period_end,cf_amount
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,8.0,Pending,3.0,2025-05-24,...,39.0,NaN,Invoice 8,EUR,CR,6110,601.70,601.70,2025-05-31,601.70
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,11.0,Pending,3.0,2025-05-24,...,49.0,NaN,Invoice 11,EUR,CR,6110,2833.28,2833.28,2025-05-31,2833.28
2,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,19.0,Pending,3.0,2025-05-24,...,88.0,NaN,Invoice 19,EUR,CR,6110,3426.53,3426.53,2025-05-31,3426.53
3,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,31.0,Pending,3.0,2025-05-24,...,41.0,NaN,Invoice 31,EUR,CR,6110,1020.02,1020.02,2025-05-31,1020.02
4,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,44.0,Pending,3.0,2025-05-24,...,58.0,NaN,Invoice 44,EUR,CR,6110,224.62,224.62,2025-05-31,224.62


In [231]:
budgeted = pd.merge(definition_acc_df,
    budgeted,
    left_on=['definition_id'],        # Columns in definition_df
    right_on=['definition_id'], # Corresponding columns in transactions_df
    how='left'
)


In [232]:
budgeted.head()

,definition_id,key,definition_type,name,id,cash_type,date,amount_LC,memo,period_end,cf_amount
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,5.0,Receipt,2025-06-01 00:00:00.000000,1000.0,Realizācija,2025-06-30,1000.0
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,6.0,Receipt,2025-06-08 00:00:00.000000,1000.0,Realizācija,2025-06-30,1000.0
2,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,7.0,Receipt,2025-06-15 00:00:00.000000,1000.0,Realizācija,2025-06-30,1000.0
3,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,8.0,Receipt,2025-06-22 00:00:00.000000,1000.0,Realizācija,2025-06-30,1000.0
4,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,9.0,Receipt,2025-06-29 00:00:00.000000,1000.0,Realizācija,2025-06-30,1000.0


In [233]:
# -------------------
# GROUP & MERGE
# -------------------
def get_period_totals(df, label):
    grouped = df.groupby(['definition_id', 'period_end', 'cash_type'])['cf_amount'].sum().unstack(fill_value=0)
    columns = ["Receipt", "Payment"]
    grouped = grouped.reindex(columns=columns, fill_value=0)
    grouped.columns = [f"{label}_{col}" for col in grouped.columns]
    return grouped

In [234]:
actual_period = get_period_totals(actual, 'Actual')
pending_period = get_period_totals(pending, 'Pending')
budgeted_period = get_period_totals(budgeted, 'Budgeted')

In [235]:
# Combine all
combined = actual_period.join(pending_period, how='outer') \
                        .join(budgeted_period, how='outer') \
                        .fillna(0).reset_index()

In [236]:
combined.head()

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment
0,2,2020-01-31,34837.53,0.0,0.0,0.0,0.0,0.0
1,2,2020-02-29,36483.82,0.0,0.0,0.0,0.0,0.0
2,2,2020-03-31,35459.50,0.0,0.0,0.0,0.0,0.0
3,2,2020-04-30,42342.85,0.0,0.0,0.0,0.0,0.0
4,2,2020-05-31,27407.06,0.0,0.0,0.0,0.0,0.0


In [237]:
# -------------------
# SPLIT PAST / FUTURE
# -------------------
today = pd.to_datetime(datetime.date.today())
this_period_end = date_offset.rollforward(today)

# Split
past = combined[combined['period_end'] < this_period_end]
future = combined[combined['period_end'] >= this_period_end]


In [238]:
# -------------------
# CASHFLOW LOGIC
# -------------------

# For past: use only actuals
past['income'] = past.get('Actual_Receipt', 0)
past['expense'] = past.get('Actual_Payment', 0)

In [239]:
past.head()

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment,income,expense
0,2,2020-01-31,34837.53,0.0,0.0,0.0,0.0,0.0,34837.53,0.0
1,2,2020-02-29,36483.82,0.0,0.0,0.0,0.0,0.0,36483.82,0.0
2,2,2020-03-31,35459.50,0.0,0.0,0.0,0.0,0.0,35459.50,0.0
3,2,2020-04-30,42342.85,0.0,0.0,0.0,0.0,0.0,42342.85,0.0
4,2,2020-05-31,27407.06,0.0,0.0,0.0,0.0,0.0,27407.06,0.0


In [240]:
# For future: use max(budgeted, actual+pending)
future['actual_plus_pending_income'] = future.get('Actual_Receipt', 0) + future.get('Pending_Receipt', 0)
future['actual_plus_pending_expense'] = future.get('Actual_Payment', 0) + future.get('Pending_Payment', 0)

future['income'] = future[['Budgeted_Receipt', 'actual_plus_pending_income']].max(axis=1)
future['expense'] = future[['Budgeted_Payment', 'actual_plus_pending_expense']].min(axis=1)



In [241]:
future.head()

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment,actual_plus_pending_income,actual_plus_pending_expense,income,expense
64,2,2025-05-31,41088.99,0.0,810680.55,0.0,0.0,0.0,851769.54,0.0,851769.54,0.0
65,2,2025-06-30,21011.12,0.0,36553.27,0.0,5000.0,0.0,57564.39,0.0,57564.39,0.0
66,2,2025-07-31,7289.03,0.0,21992.93,0.0,4000.0,0.0,29281.96,0.0,29281.96,0.0
67,2,2025-08-31,21163.81,0.0,61398.90,0.0,5000.0,0.0,82562.71,0.0,82562.71,0.0
68,2,2025-09-30,21331.78,0.0,6094.69,0.0,4000.0,0.0,27426.47,0.0,27426.47,0.0


In [242]:
# Combine
cashflow = pd.concat([past, future], ignore_index=True)
cashflow['net_cashflow'] = cashflow['income'] + cashflow['expense']

In [243]:
cashflow.head()

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment,income,expense,actual_plus_pending_income,actual_plus_pending_expense,net_cashflow
0,2,2020-01-31,34837.53,0.0,0.0,0.0,0.0,0.0,34837.53,0.0,NaN,NaN,34837.53
1,2,2020-02-29,36483.82,0.0,0.0,0.0,0.0,0.0,36483.82,0.0,NaN,NaN,36483.82
2,2,2020-03-31,35459.50,0.0,0.0,0.0,0.0,0.0,35459.50,0.0,NaN,NaN,35459.50
3,2,2020-04-30,42342.85,0.0,0.0,0.0,0.0,0.0,42342.85,0.0,NaN,NaN,42342.85
4,2,2020-05-31,27407.06,0.0,0.0,0.0,0.0,0.0,27407.06,0.0,NaN,NaN,27407.06


In [244]:
# -------------------
# CASHFLOW OUTPUT
# -------------------

# Pivot first
pivot_all_periods = cashflow.pivot_table(
    index='definition_id',
    columns='period_end',
    values='net_cashflow',
    aggfunc='sum'
).fillna(0)


In [245]:
pivot_all_periods

period_end,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,...,2025-03-31,2025-04-30,2025-05-31,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
definition_id,,,,,,,,,,,,,,,,,,,,,
2,34837.53,36483.82,35459.50,42342.85,27407.06,15389.49,20400.29,2466.67,17158.65,18168.51,...,21998.32,23706.03,851769.54,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
3,-19111.29,-9998.47,-39924.33,-14008.52,-12040.45,-23068.53,-22482.84,-15473.41,-47262.95,-22087.67,...,-22493.64,-14946.32,-1015003.76,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
4,-18549.32,-8182.26,-21393.71,-1927.84,-20380.09,-22969.65,-4716.44,-7719.77,-23473.22,-1183.03,...,-7089.59,-15492.59,-456234.97,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
5,35559.36,12768.99,32416.55,25740.03,11209.39,20690.58,32081.66,15605.56,19595.91,13887.75,...,-4170.01,14226.22,489998.67,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
8,6874.59,6525.16,1377.21,10950.75,1301.11,-2091.22,5309.37,-1075.40,-7136.12,1844.94,...,-3101.48,-2329.83,-25552.44,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73


In [246]:
# Fill missing periods and drop unnecessary

# Build full period range
min_period = date_offset.rollforward(pd.to_datetime(date_from))
max_period = date_offset.rollforward(pd.to_datetime(date_through))

report_periods = pd.date_range(start=min_period, end=max_period, freq=date_frequency)

# Reindex pivot to include all requested periods
pivot_cf = pivot_all_periods.reindex(columns=report_periods, fill_value=0)

# Sort columns just in case
pivot_cf = pivot_cf.sort_index(axis=1)

In [247]:
pivot_cf

,2025-06-30,2025-07-31,2025-08-31,2025-09-30,2025-10-31,2025-11-30,2025-12-31
definition_id,,,,,,,
2,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
3,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
4,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
5,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
8,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73


In [248]:
# Store pivot_cf for cash flow figure
graph_pivot = pd.merge(definition_acc_df, pivot_cf, on="definition_id", how="right")
graph_pivot.drop(columns=["definition_id", "key", "definition_type"], inplace=True)
graph_pivot=graph_pivot.set_index("name")

In [249]:
graph_pivot

,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
name,,,,,,,
Ieņēmumi no preču un pakalpojumu pārdošanas,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
Maksājumi piegādātājiem,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
Maksājumi darbiniekiem,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
Pārējie pamatdarbības ieņēmumi un izdevumi,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
Izdevumi nodokļu maksājumiem,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73


In [250]:
# Ensure 0 instead of NaN in empty rows
pivot_cf = pd.merge(definition_acc_df["definition_id"], pivot_cf, left_on="definition_id", right_on="definition_id", how="left").fillna(0)
pivot_cf.set_index("definition_id", inplace=True)

In [251]:
pivot_cf

,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
definition_id,,,,,,,
2,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
3,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
4,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
5,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00
12,0.00,0.00,0.00,0.00,0.00,0.00,0.00
13,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [252]:
# -----------------------------------------------------------------------------------------------------------------
# ********************** (2) Start working with subtotals based on totaled definitions *******************************
# -----------------------------------------------------------------------------------------------------------------

# Merge totals into totals definition
definition_totals_df = pd.merge(
    definition_tot_df,
    definition_totals_df,
    on="definition_id", how="left")
definition_totals_df.drop(columns=["id"], inplace=True)


In [253]:
definition_totals_df

,definition_id,key,definition_type,name,definition_summarized
0,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,2
1,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,3
2,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,4
3,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,5
4,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,2
5,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,3
6,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,4
7,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,5
8,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,7
9,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,8


In [254]:
# Merge totals definition with summarized accounts pivot

merged_totals = pd.merge(definition_totals_df,
    pivot_cf,
    left_on='definition_summarized',
    right_on='definition_id', 
    how='left'
)

# Drop unnecessary columns

merged_totals.drop(columns = ['key', 'definition_type', 'name', 'definition_summarized'], inplace=True)


In [255]:
merged_totals

,definition_id,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
0,6,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
1,6,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
2,6,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
3,6,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
4,10,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
5,10,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
6,10,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
7,10,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
8,10,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,10,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73


In [256]:
# Choose value_columns for further summarization

value_columns = [col for col in merged_totals.columns if col not in ['definition_id']]

In [257]:
value_columns

[Timestamp('2025-06-30 00:00:00'),
 Timestamp('2025-07-31 00:00:00'),
 Timestamp('2025-08-31 00:00:00'),
 Timestamp('2025-09-30 00:00:00'),
 Timestamp('2025-10-31 00:00:00'),
 Timestamp('2025-11-30 00:00:00'),
 Timestamp('2025-12-31 00:00:00')]

In [258]:
# Summarize value_columns based on group value

summarized_totals = merged_totals.groupby('definition_id', group_keys=False)[value_columns].sum()

# .reset_index()

summarized_totals.fillna(0, inplace=True)

In [259]:
summarized_totals

,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
definition_id,,,,,,,
6,56739.93,13118.29,25650.31,-25379.28,-416.72,-18580.32,53707.27
10,59711.11,10418.21,33098.96,-30206.52,386.32,-26301.68,57201.00
20,0.00,0.00,0.00,0.00,0.00,0.00,0.00
27,0.00,0.00,0.00,0.00,0.00,0.00,0.00
29,59711.11,10418.21,33098.96,-30206.52,386.32,-26301.68,57201.00


In [260]:
# ------------------------------------------------------------------------------------------------------------
# ********************** (3) Start working with balances on end of each period *******************************
# ------------------------------------------------------------------------------------------------------------



In [261]:
# Step 1. Determine periods
today = pd.to_datetime(datetime.date.today())
this_period_end = date_offset.rollforward(today)
min_period = date_offset.rollforward(pd.to_datetime(date_from))
max_period = date_offset.rollforward(pd.to_datetime(date_through))

report_periods = pd.date_range(start=min_period, end=max_period, freq=date_frequency)

In [262]:
# Step 2. Determine past balances

# Group and sort actual bank balance by period
actual_cash_by_period = cash.groupby('period_end')['cf_amount'].sum().sort_index()
actual_cumulative_cash = actual_cash_by_period.cumsum()

In [263]:
actual_cumulative_cash

period_end
2020-01-31     39610.87
2020-02-29     77208.11
2020-03-31     85143.34
2020-04-30    148240.62
2020-05-31    155737.63
                ...    
2025-08-31    102471.37
2025-09-30     59616.66
2025-10-31     71306.69
2025-11-30     93681.26
2025-12-31    160331.66
Name: cf_amount, Length: 72, dtype: float64

In [264]:
if max_period >= this_period_end:
    past_balances = actual_cumulative_cash[actual_cumulative_cash.index < this_period_end]
else:
    past_balances = actual_cumulative_cash[actual_cumulative_cash.index <= max_period]



In [265]:
past_balances

period_end
2020-01-31     39610.87
2020-02-29     77208.11
2020-03-31     85143.34
2020-04-30    148240.62
2020-05-31    155737.63
                ...    
2024-12-31     53257.17
2025-01-31     93054.80
2025-02-28     98845.83
2025-03-31     83989.40
2025-04-30     89152.92
Name: cf_amount, Length: 64, dtype: float64

In [266]:
# Step 3. Determine future balances

if min_period >= this_period_end:
    future_periods = pd.date_range(start=this_period_end, end=max_period, freq=date_frequency)
else:
    future_periods = report_periods[report_periods >= this_period_end]

In [267]:
future_periods

DatetimeIndex(['2025-05-31', '2025-06-30', '2025-07-31', '2025-08-31',
               '2025-09-30', '2025-10-31', '2025-11-30', '2025-12-31'],
              dtype='datetime64[ns]', freq='ME')

In [268]:
last_actual_balance = past_balances.iloc[-1] if not past_balances.empty else 0

In [269]:
last_actual_balance

89152.92

In [270]:
# Future cashflows from pivot_all_periods
cashflow_by_period = pivot_all_periods.sum(axis=0)

bal = last_actual_balance
future_balances = {}
for period in future_periods:
    bal += cashflow_by_period.get(period, 0)
    future_balances[period] = bal



In [271]:
future_balances

{Timestamp('2025-05-31 00:00:00'): -65870.04000000002,
 Timestamp('2025-06-30 00:00:00'): -6158.930000000029,
 Timestamp('2025-07-31 00:00:00'): 4259.279999999966,
 Timestamp('2025-08-31 00:00:00'): 37358.23999999996,
 Timestamp('2025-09-30 00:00:00'): 7151.719999999976,
 Timestamp('2025-10-31 00:00:00'): 7538.039999999983,
 Timestamp('2025-11-30 00:00:00'): -18763.640000000018,
 Timestamp('2025-12-31 00:00:00'): 38437.36}

In [272]:
# Step 4. Combine past and future into all closing balance series, adjust periods
all_closing_balances = pd.Series(dtype=float)
all_closing_balances = pd.concat([
    past_balances,
    pd.Series(future_balances)
]).reindex(report_periods, method="ffill")


In [273]:
all_closing_balances

2025-06-30    -6158.93
2025-07-31     4259.28
2025-08-31    37358.24
2025-09-30     7151.72
2025-10-31     7538.04
2025-11-30   -18763.64
2025-12-31    38437.36
Freq: ME, dtype: float64

In [274]:
# Step 5. Formating

# Create a new DataFrame with closing balances only
balances = pd.DataFrame(
    [all_closing_balances],  # one row
    index=[0]  # index = 0 
)

In [275]:
# Store balances for use in Graph
graph_balances = all_closing_balances.values

In [276]:
graph_balances

array([ -6158.93,   4259.28,  37358.24,   7151.72,   7538.04, -18763.64,
        38437.36])

In [277]:


# Create a new DataFrame with repeated rows for each definition_id
balances = pd.merge(definition_bal_df["definition_id"], balances, how='cross')
balances = balances.set_index("definition_id")


In [278]:
balances

,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
definition_id,,,,,,,
30,-6158.93,4259.28,37358.24,7151.72,7538.04,-18763.64,38437.36


In [279]:
# ---------------------------------------------------------------------------------------------------------------------------------------
# ********************** (4) Put together summarized accounts, totals and balances on end of each period *******************************
# ---------------------------------------------------------------------------------------------------------------------------------------



In [280]:
pivot_cf

,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
definition_id,,,,,,,
2,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
3,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
4,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
5,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00
12,0.00,0.00,0.00,0.00,0.00,0.00,0.00
13,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [281]:
summarized_totals

,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
definition_id,,,,,,,
6,56739.93,13118.29,25650.31,-25379.28,-416.72,-18580.32,53707.27
10,59711.11,10418.21,33098.96,-30206.52,386.32,-26301.68,57201.00
20,0.00,0.00,0.00,0.00,0.00,0.00,0.00
27,0.00,0.00,0.00,0.00,0.00,0.00,0.00
29,59711.11,10418.21,33098.96,-30206.52,386.32,-26301.68,57201.00


In [282]:
balances

,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
definition_id,,,,,,,
30,-6158.93,4259.28,37358.24,7151.72,7538.04,-18763.64,38437.36


In [283]:
# Concatenate it all together
report = pd.concat([pivot_cf, summarized_totals, balances])

In [284]:
report

,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
definition_id,,,,,,,
2,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
3,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
4,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
5,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00
12,0.00,0.00,0.00,0.00,0.00,0.00,0.00
13,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [285]:
# -------------------------------------
# Prepare report for visual appearance 
# -------------------------------------

# Merge report definition header with report
report = pd.merge(definition_df, report, left_on="definition_id", right_on="definition_id", how="left")

# Sort based on key value
report.sort_values("key", inplace=True)


In [286]:
report

,definition_id,key,definition_type,name,2025-06-30 00:00:00,2025-07-31 00:00:00,2025-08-31 00:00:00,2025-09-30 00:00:00,2025-10-31 00:00:00,2025-11-30 00:00:00,2025-12-31 00:00:00
0,1,100,NaN,PAMATDARBĪBAS NAUDAS PLŪSMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
2,3,102,1.0,Maksājumi piegādātājiem,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
3,4,103,1.0,Maksājumi darbiniekiem,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
5,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,56739.93,13118.29,25650.31,-25379.28,-416.72,-18580.32,53707.27
6,7,201,1.0,Izdevumi procentu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,8,202,1.0,Izdevumi nodokļu maksājumiem,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73
8,9,203,1.0,Naudas plūsma no ārkārtas posteņiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,59711.11,10418.21,33098.96,-30206.52,386.32,-26301.68,57201.00


In [287]:
# Subtract report formatting in separate dataframe
format_df = report["definition_type"]

In [288]:
format_df

0     NaN
1     1.0
2     1.0
3     1.0
4     1.0
5     2.0
6     1.0
7     1.0
8     1.0
9     2.0
10    NaN
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    2.0
20    NaN
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    2.0
27    1.0
28    2.0
29    3.0
Name: definition_type, dtype: float64

In [289]:
# Prepare report_df for visual appearance

report.drop(columns=["definition_id", "key", "definition_type"], inplace=True)
report.set_index("name", inplace=True)

# Format column headers to show only the date part
report.columns = [col.strftime(date_format()) if not pd.isnull(col) else col for col in report.columns]

In [290]:
report

,"30. Jun, 2025","31. Jul, 2025","31. Aug, 2025","30. Sep, 2025","31. Oct, 2025","30. Nov, 2025","31. Dec, 2025"
name,,,,,,,
PAMATDARBĪBAS NAUDAS PLŪSMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ieņēmumi no preču un pakalpojumu pārdošanas,57564.39,29281.96,82562.71,27426.47,52050.86,49121.95,49292.13
Maksājumi piegādātājiem,-35832.25,-47552.46,-46088.68,-33903.15,-56715.98,-65041.92,-59832.57
Maksājumi darbiniekiem,-36795.84,-21104.60,-17142.52,-38304.87,-42803.52,-35165.69,-20230.86
Pārējie pamatdarbības ieņēmumi un izdevumi,71803.63,52493.39,6318.80,19402.27,47051.92,32505.34,84478.57
BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,56739.93,13118.29,25650.31,-25379.28,-416.72,-18580.32,53707.27
Izdevumi procentu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Izdevumi nodokļu maksājumiem,2971.18,-2700.08,7448.65,-4827.24,803.04,-7721.36,3493.73
Naudas plūsma no ārkārtas posteņiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00
